In [3]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt
from keras.models import load_model 


In [8]:
FOCUS_SOURCE_DIR = "dataset_2/focus/"
UNFOCUS_SOURCE_DIR = "dataset_2/unfocus/"
#DROWSY_SOURCE_DIR = "dataset_2/drowsy/"

TRAINING_DIR = 'images_2/training/'
VALIDATION_DIR = 'images_2/validation/'

TRAINING_FOCUS_DIR = os.path.join(TRAINING_DIR, "focus/")
VALIDATION_FOCUS_DIR = os.path.join(VALIDATION_DIR, "focus/")

TRAINING_UNFOCUS_DIR = os.path.join(TRAINING_DIR, "unfocus/")
VALIDATION_UNFOCUS_DIR = os.path.join(VALIDATION_DIR, "unfocus/")

#TRAINING_DROWSY_DIR = os.path.join(TRAINING_DIR, "drowsy/")
#VALIDATION_DROWSY_DIR = os.path.join(VALIDATION_DIR, "drowsy/")


In [9]:
# Your function should perform copies rather than moving images so original directories should contain unchanged images
print(f"Original focus's directory has {len(os.listdir(FOCUS_SOURCE_DIR))} images")
print(f"Original unfocus's directory has {len(os.listdir(UNFOCUS_SOURCE_DIR))} images")
#print(f"Original drowsy's directory has {len(os.listdir(DROWSY_SOURCE_DIR))} images\n")

# Training and validation splits
print(f"There are {len(os.listdir(TRAINING_FOCUS_DIR))} images of focus for training")
print(f"There are {len(os.listdir(TRAINING_UNFOCUS_DIR))} images of unfocus for training")
#print(f"There are {len(os.listdir(TRAINING_DROWSY_DIR))} images of drowsy for training\n")

print(f"There are {len(os.listdir(VALIDATION_FOCUS_DIR))} images of focus for validation")
print(f"There are {len(os.listdir(VALIDATION_UNFOCUS_DIR))} images of unfocus for validation")
#print(f"There are {len(os.listdir(VALIDATION_DROWSY_DIR))} images of drowsy for validation")

Original focus's directory has 12870 images
Original unfocus's directory has 12870 images
There are 9009 images of focus for training
There are 9009 images of unfocus for training
There are 3861 images of focus for validation
There are 3861 images of unfocus for validation


In [10]:
# grader-required-cell

# GRADED FUNCTION: train_val_generators
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):

  train_datagen = ImageDataGenerator(rescale=1.0/255.0)

  # Pass in the appropriate arguments to the flow_from_directory method
  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=32,
                                                      class_mode='binary', #categorical
                                                      target_size=(224, 224)
                                                      )

  # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
  validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

  # Pass in the appropriate arguments to the flow_from_directory method
  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=32,
                                                                class_mode='binary', # categorical
                                                                target_size=(224, 224)
                                                                )
  ### END CODE HERE
  return train_generator, validation_generator

In [11]:
train_generator, validation_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR)

Found 18018 images belonging to 2 classes.
Found 7722 images belonging to 2 classes.


In [13]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


58889256/58889256 [==============================] - 128s 2us/step


In [14]:
for layer in base_model.layers:
    layer.trainable = False


In [15]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Optional fully connected layer
predictions = Dense(1, activation='sigmoid')(x)  # Output layer for binary classification

model = Model(inputs=base_model.input, outputs=predictions)


In [16]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [18]:
with tf.device('/GPU:0'):
    history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator)


Epoch 1/10
 66/564 [==>...........................] - ETA: 1:55 - loss: 0.6284 - accuracy: 0.6610

In [ ]:
#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.show()
print("")

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.show()